In [7]:
import pandas as pd
df = pd.read_csv("/Users/lhe/Desktop/438/bitcoin/bitcoin_history.csv")
print(df.head())
print(df.tail())
print(df.columns)



           Date       Open       High        Low      Close  Adj Close  \
0  Feb 20, 2025  96,628.40  98,722.29  96,481.41  98,347.95  98,347.95   
1  Feb 19, 2025  95,532.53  96,855.59  95,011.97  96,635.61  96,635.61   
2  Feb 18, 2025  95,773.81  96,695.38  93,388.84  95,539.55  95,539.55   
3  Feb 17, 2025  96,179.01  97,032.23  95,243.55  95,773.38  95,773.38   
4  Feb 16, 2025  97,580.49  97,725.59  96,060.98  96,175.03  96,175.03   

           Volume  
0  31,327,236,096  
1  28,990,872,862  
2  37,325,720,482  
3  27,336,550,690  
4  16,536,755,396  
              Date    Open    High     Low   Close Adj Close      Volume
3805  Sep 21, 2014  408.08  412.43  393.18  398.82    398.82  26,580,100
3806  Sep 20, 2014  394.67  423.30  389.88  408.90    408.90  36,863,600
3807  Sep 19, 2014  424.10  427.83  384.53  394.80    394.80  37,919,700
3808  Sep 18, 2014  456.86  456.86  413.10  424.44    424.44  34,483,200
3809  Sep 17, 2014  465.86  468.17  452.42  457.33    457.33  21,056,8

In [8]:
if not pd.api.types.is_datetime64_any_dtype(df['Date']):
    df['Date'] = pd.to_datetime(df['Date'])
    df = df.set_index('Date')
    df = df.sort_index()
print(df.isna().sum())
# remove commas from numeric columns and convert to float
for col in ["Open", "High", "Low", "Close", "Adj Close", "Volume"]:
    df[col] = df[col].astype(str).str.replace(",", "").astype(float)



Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64


In [9]:
df['ret_1d'] = df['Close'].pct_change()
df['sma_7'] = df['Close'].rolling(7).mean()
df['sma_30'] = df['Close'].rolling(30).mean()
df['sma_diff'] = df['sma_7'] - df['sma_30']
df['volatility_7d'] = df['ret_1d'].rolling(7).std()
df['target'] = df['ret_1d'].shift(-1)  #next day return 
df = df.dropna()


In [11]:
import numpy as np
from sklearn.model_selection import train_test_split

# 选择特征列（可以先用简单三四个）
feature_cols = ['ret_1d', 'sma_diff', 'volatility_7d']

X = df[feature_cols]
y = df['target']


In [12]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, shuffle=False
)

print(X_train.shape, X_test.shape)


(3024, 3) (756, 3)


In [13]:
import xgboost as xgb

model = xgb.XGBRegressor(
    n_estimators=300,
    max_depth=3,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    objective='reg:squarederror',
    random_state=42
)

model.fit(X_train, y_train)

y_pred = model.predict(X_test)


XGBoostError: 
XGBoost Library (libxgboost.dylib) could not be loaded.
Likely causes:
  * OpenMP runtime is not installed
    - vcomp140.dll or libgomp-1.dll for Windows
    - libomp.dylib for Mac OSX
    - libgomp.so for Linux and other UNIX-like OSes
    Mac OSX users: Run `brew install libomp` to install OpenMP runtime.

  * You are running 32-bit Python on a 64-bit OS

Error message(s): ["dlopen(/usr/local/lib/python3.11/site-packages/xgboost/lib/libxgboost.dylib, 0x0006): Library not loaded: '@rpath/libomp.dylib'\n  Referenced from: '/usr/local/lib/python3.11/site-packages/xgboost/lib/libxgboost.dylib'\n  Reason: tried: '/usr/local/opt/libomp/lib/libomp.dylib' (no such file), '/usr/local/opt/libomp/lib/libomp.dylib' (no such file), '/usr/local/Cellar/python@3.11/3.11.4_1/Frameworks/Python.framework/Versions/3.11/Resources/Python.app/Contents/MacOS/../../../../../../../../../../../lib/libomp.dylib' (no such file), '/usr/local/lib/libomp.dylib' (no such file), '/usr/lib/libomp.dylib